In [1]:
import pandas as pd
import os

In [2]:
text_num_mapping_start = 1
model_test_run = False #This reduces the data generated by selecting significantly lesser number of sessions to test the model
remove_consecutive_identical_clicks = False
#Todo implement identical consecutive clicks - only do that if the timedelta between clicks is quite less- otherwise it doesnt make sense.
sequence_augmentation = True
carsi_labels_only = True
regenerate_context_data = False
whole_session_context = False
data_autoencoder = False
pad_to_window_size = True
window = 100 #seconds

base_path = '../datasets/sequential/'
combined_context_path = '../data/05_Interaction_Sequences/context.csv'
augmentation_folder = 'aug/' if sequence_augmentation else 'non_aug/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'
sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters'
sequence_path = f'{base_path}{augmentation_folder}seq'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'

In [3]:
dc_train = pd.read_csv(train_dynamic_context_path)
dc_test = pd.read_csv(test_dynamic_context_path)

sc_train = pd.read_csv(train_static_context_path)
sc_test = pd.read_csv(test_static_context_path)

seq_train = pd.read_csv(sequence_path+'/train.tsv', sep='\t')
seq_test = pd.read_csv(sequence_path+'/test.tsv', sep='\t')

In [4]:
# duplicates_seq = seq_train[seq_train[['item_id_seq_train', 'item_id_target']].duplicated()]
# duplicates_seq.sort_values(by=['item_id_seq_train', 'item_id_target']).to_csv('duplicate_seq.csv', index=False)

# remove all the consecutive duplicate clicks in a sample
# remove all the duplicate rows of clicks in the entire df  
#todo fix timedelta. right now the transformation for the time delta is ignored.

In [5]:
def remove_consecutive_duplicates(input_list):
    result = []
    removed_indices = []  # List to store indices of removed elements
    prev = None
    for i, num in enumerate(input_list):
        if num != prev:
            result.append(num)
        else:
            removed_indices.append(i)
        prev = num
    return result, removed_indices

def join_list(lst):
    return ' '.join(map(str, lst))

def create_zeros_list(lst):
    return [0] * len(lst)
    
def seq_duplicate_remove(df):
    df['interaction_time_delta_train_list'] = df['interaction_time_delta_train'].str.split().apply(lambda x: [int(i) for i in x])
    df['seq_list'] = df['item_id_seq_train'].str.split().apply(lambda x: remove_consecutive_duplicates([int(i) for i in x]))
    # Apply the function to each row of the "item_id_seq_train" column
    result = seq_train['item_id_seq_train'].str.split().apply(lambda x: remove_consecutive_duplicates([int(i) for i in x]))

    # Extract cleaned list and removed indices into separate columns
    df['seq_list'] = result.apply(lambda x: x[0])
    df['removed_indices'] = result.apply(lambda x: x[1])
    
    df['seq_non_dup'] = df['seq_list'].apply(join_list)

    max_length = df['seq_list'].apply(lambda x: len(x)).max()
    print("Maximum length of lists in seq_list column:", max_length)

    empty_lists_exist = any(df['seq_list'].apply(lambda x: len(x) == 0))
    if empty_lists_exist:
        print("There are empty lists in the 'seq_list' column.")
    else:
        print("There are no empty lists in the 'seq_list' column.")

    df['wrong_time_delta_interaction_list'] = df['seq_list'].apply(create_zeros_list)
    df['wrong_time_delta_interaction'] = df['wrong_time_delta_interaction_list'].apply(join_list)

    df.drop(columns=['seq_list', 'item_id_seq_train'], inplace=True)
    df.rename(columns={'seq_non_dup': 'item_id_seq_train'}, inplace=True)

    desired_column_order = ['window_id', 'item_id_seq_train', 'item_id_target', 'wrong_time_delta_interaction', 'session']
    df = df[desired_column_order]
    return df

In [6]:
seq_train_updated = seq_duplicate_remove(seq_train)
seq_test_updated = seq_duplicate_remove(seq_test)

seq_test_updated.to_csv(os.path.join(sequence_path, 'non_dup' ,'test.tsv'), sep='\t', index=False)
seq_train_updated.to_csv(os.path.join(sequence_path, 'non_dup' ,'train.tsv'), sep='\t', index=False)

Maximum length of lists in seq_list column: 53
There are no empty lists in the 'seq_list' column.
Maximum length of lists in seq_list column: 28
There are no empty lists in the 'seq_list' column.


In [7]:
# def split_consecutive(numbers):
#     if not numbers:
#         return []

#     # Initialize the list of lists to store consecutive sequences
#     result = [[numbers[0]]]

#     # Iterate through the numbers
#     for num in numbers[1:]:
#         # If the current number is consecutive to the last number in the last sublist
#         if num == result[-1][-1] + 1:
#             result[-1].append(num)  # Add it to the current sublist
#         else:
#             result.append([num])  # Otherwise, start a new sublist

#     return result


# def update_time_delta(row):
#     time_delta_list = row['interaction_time_delta_train_list']
    
#     removed_indices = row['removed_indices']
#     # print(time_delta_list)
#     # print(removed_indices)
#     updated_time_delta = []  # Initialize updated_time_delta variable
    
#     split = split_consecutive(removed_indices)

#     if len(removed_indices) != 0:
#         # for i in range(1, len(time_delta_list)):
#         #     if i not in removed_indices:
#         #         updated_time_delta.append(time_delta_list[i])
#         #     else:
#         #         new_time_delta = time_delta_list[i] + time_delta_list[i-1]
#         #         updated_time_delta.append(new_time_delta)
#         for count, i in enumerate(split):
#             print('count', count)
#             if count == 0:
#                 print('inside count =0', i)
#                 new_time_delta = time_delta_list[i[0]-1]
#             else:
#                 print('i am here')
#                 print(i)
#                 print(new_time_delta)
#                 new_time_delta = updated_time_delta[-1]
#                 print(new_time_delta)
#             for j in i:
#                 print('inside for loop',j)
#                 new_time_delta = new_time_delta + time_delta_list[j]
#             updated_time_delta.append(new_time_delta)
#     else:
#         updated_time_delta = time_delta_list  # Assign original time_delta_list
    
#     return updated_time_delta  # Return the updated list

# seq_train_updated['updated_interaction_time_delta'] = seq_train_updated[['interaction_time_delta_train_list', 'removed_indices']].apply(update_time_delta, axis=1)